In [1]:
import pandas as pd
import numpy as np
from hyperopt import fmin, hp
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder

In [2]:
train = pd.read_csv("sets_de_datos/train.csv", index_col=0)

In [3]:
train["fecha"] = pd.to_datetime(train["fecha"])
train["fecha_n"] = train["fecha"].dt.year * 10000 + train["fecha"].dt.month * 100 + train["fecha"].dt.day

In [4]:
col_borrar = ["direccion", "lat", "lng", "titulo", "descripcion", "idzona", "fecha"]
train_1 = train.drop(col_borrar, axis=1)

In [5]:
train_1 = train_1.fillna(value = {'tipodepropiedad' : 'nan', 'provincia' : 'nan', 'ciudad' : 'nan'})

for c in ["antiguedad", "metrostotales", "metroscubiertos"]:
    imputer = SimpleImputer(strategy="mean")
    train_1[c] = imputer.fit_transform(train_1[[c]])

for c in ["habitaciones", "banos", "garages"]:
    imputer = SimpleImputer(strategy="constant", fill_value=0)
    train_1[c] = imputer.fit_transform(train_1[[c]])

In [6]:
col_categoricas = ["provincia", "ciudad", "tipodepropiedad"]
col_datos = train_1.dtypes[train.dtypes != "object"].index

In [7]:

codificador = LabelEncoder()
train_codificado = train_1[col_categoricas].apply(codificador.fit_transform)
    

In [8]:
train_2 = train_1[col_datos].join(train_codificado)

In [9]:
datos = train_2.drop(["precio"], axis=1)
precio = train_2["precio"]

## ========================Empiezo con hyperopt========================

In [10]:
from xgboost import XGBRegressor
import xgboost as xgb

In [19]:
from sklearn.model_selection import cross_val_score

In [15]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

#### Empiezo con un ejemplo facilito para familiarizarme

In [21]:
def hyperopt_train_test(params):
    modelo = XGBRegressor(**params)
    return cross_val_score(modelo, datos, precio).mean()

In [22]:
space_xgb = {
    "objective" : "reg:squarederror",
    "subsample" : hp.uniform("subsample", 0, 1),
    "n_estimators" : hp.choice("n_estimators", range(1000, 2001)),
    "min_child_weight" : hp.choice("min_child_weight", range(1,6)),
    "max_depth" : hp.choice("max_depth", range(3, 11)),
    "learning_rate" : hp.uniform("learning_rate", 0.01, 0.2),
    "reg_lambda" : hp.uniform("reg_lambda", 0.03, 0.3),
    "gamma" : hp.uniform("gamma", 0, 0.5),
    "colsample_bytree" : hp.uniform("colsample_bytree", 0.5, 1),
    "subsample" : hp.uniform("subsample", 0.5, 1)
    
}

In [23]:
def f(params):
    acc = hyperopt_train_test(params)
    return {'loss': -acc, 'status': STATUS_OK}

In [24]:
trials = Trials()

In [25]:
best = fmin(f, space_xgb, algo=tpe.suggest, max_evals=2, trials=trials)

  0%|          | 0/2 [00:00<?, ?it/s, best loss: ?]

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)

/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \

/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \

/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \



 50%|█████     | 1/2 [29:36<29:36, 1776.68s/it, best loss: -0.7951106208271902]

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)

/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \

/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \

/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \



100%|██████████| 2/2 [47:45<00:00, 1432.73s/it, best loss: -0.7951106208271902]
